<a href="https://colab.research.google.com/github/pulseq/MR-Physics-with-Pulseq/blob/main/tutorials/06_spin_echos/notebooks/Spin_Echo_Exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spin Echo Exercise 1

The pulseq example shown below generates a sequence of rf-pulses spaced at integer time intervals dtp.
Maximize the number of echoes generated by 4 pulses by modifying the time intervals dtp between successive pulses.
How many signals can maximally occur after the last pulse ?
What is the minimum total time necessary to create all possible signals ?


If you want to execute the sequence on the scanner with high flip angles, you may have to increase the time unit T0 and/or reduce the time_bw_product of the pulse. Can you guess, why it is set rather high ?


In [ ]:
try:
  import pypulseq as mr
  print("pypulseq package is readily available\n")

except ImportError or ModuleNotFoundError:
  !pip install git+https://github.com/imr-framework/pypulseq.git

In [ ]:
import math
import warnings
import numpy as np
from matplotlib import pyplot as plt

import pypulseq as mr

Thats my sequence

In [ ]:
system = mr.Opts(
max_grad=32,
    grad_unit="mT/m",
    max_slew=130,
    slew_unit="T/m/s",
    rf_ringdown_time=100e-6,
    rf_dead_time=100e-6,
    adc_dead_time=10e-6,
)

seq = mr.Sequence(system)           # Create a new sequence object
                                       # Define FOV and resolution
dtp = np.array([1, 3, 3, 7])
tp=np.cumsum(dtp)

flip = np.array([10, 20, 30, 40])
slice_thickness = 50e-3            # slice
                                 # repetition time TR
T0 = 1e-3                        # echo time T0
Nspoi_x = 5
Nspoi_y = 7
dGt = 100e-6
# Create alpha-degree slice selection pulse and gradient
rf, gz, _ = mr.make_sinc_pulse(
  flip_angle = np.pi / 180,
  duration = T0-2*dGt,
  delay = dGt,
  slice_thickness = slice_thickness,
  apodization = 0.42,
  time_bw_product = 10,
  system = system,
  return_gz=True,
)
rf_ref=rf.signal
gz.rise_time = dGt
gz.fall_time = dGt
gz.delay = 0

Nx = gz.flat_time / system.grad_raster_time
adc = mr.make_adc(
  num_samples = Nx,
  duration = gz.flat_time,
  delay=dGt,
  system=system,
)

# gradient spoiling
gx_spoil= mr.make_trapezoid(
  channel="x",
  area=Nspoi_x * gz.area,
  system=system,
)
gy_spoil= mr.make_trapezoid(
  channel="y",
  area=Nspoi_y * gz.area,
  system=system,
)
rf_phase=0
rf_inc=0

for nump in range (len(dtp)):
    rf.signal = rf_ref * flip[nump]
    seq.add_block(rf, gz)

    for ne in range (dtp[nump]-1):
    	seq.add_block(gz, adc)


[k_traj_adc, k_traj, t_excitation, t_refocusing, t_adc] = seq.calculate_kspace() # TODO: check whether calculate_kspacePP() is better

seq.write('echoes.seq')       # Write to pulseq file
seq.plot()
print(tp)
#seq.install('siemens')


Plot Phasegraph

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def pg_plot(tp, gm):
    tp=tp.tolist()
    tpend=2*(tp[-1])
    print(tpend)
    tp.append(tpend)
    npulses = len(tp)
    pg = np.zeros((npulses, 3**(npulses-1)))
    pg[0, 0] = 0
    npg = 1
    m = 0

    for k in range(1, 2):
        pg[k, m] = pg[k-1, 0]
        kk = 1
        if k < npulses:
            plt.plot(tp[k], pg[k, m], 'ok', markerfacecolor='k', markersize=5)
        pg[k, m] = pg[k-1, kk] + (tp[k] - tp[k-1]) * gm
        plt.plot([tp[k-1], tp[k]], [pg[k-1, kk], pg[k, m]], 'b-')
        m += 1
        pg[k, m] = -pg[k, m-1]
        m += 1
        plt.plot([tp[k], tp[k]], [-gm * tp[npulses-1] + gm, gm * tp[npulses-1] + gm], 'r-')

    npg = 3
    for k in range(2, npulses):
        m = 0
        for kk in range(1, npg+1):
            pg[k, m] = pg[k-1, kk-1]
            plt.plot([tp[k-1], tp[k]], [pg[k-1, kk-1], pg[k, m]], 'c-')
            m += 1
            if pg[k-1, kk-1] != 0.1:
                pg[k, m] = pg[k-1, kk-1] + (tp[k] - tp[k-1]) * gm
                plt.plot([tp[k-1], tp[k]], [pg[k-1, kk-1], pg[k, m]], 'b-')
            m += 1
            pg[k, m] = -pg[k, m-1]
            m += 1

        plt.plot([0, tp[npulses-1]], [0, 0], 'k', linewidth=2)
        plt.plot([tp[k], tp[k]], [-gm * tp[npulses-1] + gm, gm * tp[npulses-1] + gm], 'r-')
        npg *= 3
        m = 0
        nd = 1
        pgc = np.sort(pg[k, 0:npg])

        pgv = [pgc[0]]

        for kk in range(1, npg):
            for knd in range(1, nd+1):
                if pgc[kk] != pgv[nd-1]:
                    nd += 1

                    pgv.append(pgc[kk])

        npg = nd
        pg[k, :] = 0
        pg[k, 0:npg] = pgv

    plt.axis([0, tp[npulses-1], np.min(pg), -np.min(pg)])
    plt.show()

# Example usage:

#tp = np.array([0, 1, 4, 13, 40])
gm = 0.5
pg_plot(tp, gm)